In [1]:
import avro.schema
import confluent_kafka.avro

In [3]:
def connect_kafka_producer(value_schema, key_schema, 
                           server='localhost:29092', schema_registry = 'http://localhost:8081', api=True):
    """
    Create the kafka producer with avro schema 
    :param value_schema: schema for the message 'string'
    :param key_schema: schema for the key 'string'
    :return: a kafka producer
    """
    producer = None
    try:
        producer = confluent_kafka.avro.AvroProducer(
    {
        'api.version.request': api,
        'compression.codec': 'gzip',
        'default.topic.config': {'request.required.acks': 1},
        'bootstrap.servers': server,
        'schema.registry.url': schema_registry,
    }, 
    default_key_schema=avro.schema.Parse(value_schema), 
    default_value_schema=avro.schema.Parse(key_schema))
        
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return producer

In [12]:
def connect_kafka_consummer(topic, server='localhost:29092', schema_registry = 'http://localhost:8081', api=True):
    conf = {'group.id': 'mygroup', 
            'api.version.request': api,
            'default.topic.config': {'auto.offset.reset': 'smallest'},
            'bootstrap.servers': server,
            'schema.registry.url': schema_registry
           }
    
    consumer = None
    try:
        consumer = confluent_kafka.avro.AvroConsumer(conf)
        consumer.subscribe([topic])
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return consumer

In [4]:
producer = connect_kafka_producer('{"type": "string"}', '{"type": "string"}')

In [5]:
producer.produce(topic='bug', value='not empty', key='test key')
producer.flush() #must be used before exiting; it is a synchronous producer and it kills performance if we use after each produce

0

In [13]:
consumer = connect_kafka_consummer('bug')

In [14]:
msg = consumer.poll()

In [15]:
msg.value()

'not empty'

In [16]:
msg.key()

'test key'